In [4]:
import pandas as pd

Get data

In [5]:
!wget https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv

--2022-11-16 07:06:58--  https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic.csv.1’

titanic.csv.1       100%[===================>]  58.89K  --.-KB/s    in 0.008s  

2022-11-16 07:06:58 (7.10 MB/s) - ‘titanic.csv.1’ saved [60302/60302]



Keterangan data <br>
Class Survived (target)
* 0 = No
* 1 = Yes

In [6]:
df=pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Separate the data nad the target

In [7]:
target="Survived"
target=df[target]
target

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [8]:
feature=df.drop(columns=["Survived","PassengerId"])
feature.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Check samples features

In [9]:
print(f'The feture contains : {feature.shape[0]} samples and {feature.shape[1]} feeatures')

The feture contains : 891 samples and 10 feeatures


In [10]:
feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


## Split data dtype 
* Dtype onject and data non object(numeric)

In [11]:
from sklearn.compose import make_column_selector
feature_numeric=make_column_selector(dtype_exclude='object')
feature_nonNumeric=make_column_selector(dtype_include='object')

In [12]:
feature_numeric=feature[feature_numeric]
feature_numeric.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [13]:
feature_nonNumeric=feature[feature_nonNumeric]
feature_nonNumeric.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,NaN,S


# Preprocessing data
* Standard scaler for numeric
* Hot encoder for non numeric

### Use standard scaler for numeric

In [14]:
feature_numeric.describe()

,Pclass,Age,SibSp,Parch,Fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,20.125000,0.000000,0.000000,7.910400
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,38.000000,1.000000,0.000000,31.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200


In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(feature_numeric)

StandardScaler()

[Perbedaan](https://datascience.stackexchange.com/questions/12321/whats-the-difference-between-fit-and-fit-transform-in-scikit-learn-models#:~:text=In%20summary%2C%20fit%20performs%20the,in%20one%20possibly%20optimized%20step.)
* `.fit`
* `.transform`
* `.fit_transform` <br>


In [17]:
scaler.fit_transform(feature_numeric)

array([[ 0.82737724, -0.53037664,  0.43279337, -0.47367361, -0.50244517],
       [-1.56610693,  0.57183099,  0.43279337, -0.47367361,  0.78684529],
       [ 0.82737724, -0.25482473, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [ 0.82737724,         nan,  0.43279337,  2.00893337, -0.17626324],
       [-1.56610693, -0.25482473, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.82737724,  0.15850313, -0.4745452 , -0.47367361, -0.49237783]])

In [18]:
scaler.transform(feature_numeric)

array([[ 0.82737724, -0.53037664,  0.43279337, -0.47367361, -0.50244517],
       [-1.56610693,  0.57183099,  0.43279337, -0.47367361,  0.78684529],
       [ 0.82737724, -0.25482473, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [ 0.82737724,         nan,  0.43279337,  2.00893337, -0.17626324],
       [-1.56610693, -0.25482473, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.82737724,  0.15850313, -0.4745452 , -0.47367361, -0.49237783]])

In [16]:
scaler.scale_

array([ 0.83560193, 14.51632115,  1.10212444,  0.80560476, 49.66553444])

### Standard data for categorical

In [23]:
feature_nonNumeric.count().sort_index()

Cabin       204
Embarked    889
Name        891
Sex         891
Ticket      891
dtype: int64

In [24]:
target.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [25]:
print(f"The dataset is composed of {feature_nonNumeric.shape[1]} features")

The dataset is composed of 5 features


## Use OrdinalEncoder 
`from sklearn.preprocessing import OrdinalEncoder`

In [28]:
feature_nonNumeric.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,NaN,S


In [35]:
target_encode=feature_nonNumeric[["Sex"]]

In [84]:
target_encode

,Sex
0,male
1,female
2,female
3,female
4,male
...,...
886,male
887,female
888,female
889,male


In [36]:
from sklearn.preprocessing import OrdinalEncoder

In [39]:
encoder = OrdinalEncoder()
sex_encoded = encoder.fit_transform(target_encode)
sex_encoded[:5]

array([[1.],
       [0.],
       [0.],
       [0.],
       [1.]])

In [40]:
encoder.categories_

[array(['female', 'male'], dtype=object)]

In [51]:
print(f'Total feature encoded : {encoder.n_features_in_}')

Total feature encoded : 5


Value atau nilai dari kolom sex yang dilakukan encoder

In [42]:
encoder = OrdinalEncoder()
data_encoded = encoder.fit_transform(feature_nonNumeric)
data_encoded[:5]

array([[108.,   1., 523.,  nan,   2.],
       [190.,   0., 596.,  81.,   0.],
       [353.,   0., 669.,  nan,   2.],
       [272.,   0.,  49.,  55.,   2.],
       [ 15.,   1., 472.,  nan,   2.]])

In [53]:
data_encoded.dtype

dtype('float64')

Dapat dilihat bahwa `OrdinalEncoder` output keluaran menghasilkan tipe data 
`float64`  <br>
Akan tetapi OrdinalEncoder menghasilkan angka dari 0-n 
[Lihat](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features) 

In [45]:
print(
    f"The dataset encoded contains {data_encoded.shape[1]} features")

The dataset encoded contains 5 features


### Encode use Onehot
`from sklearn.preprocessing import OneHotEncoder`

In [54]:
from sklearn.preprocessing import OneHotEncoder

In [90]:
encode_oneHot=OneHotEncoder()
sex_encoded_sparseTrue = encode_oneHot.fit_transform(target_encode)
sex_encoded_sparseTrue.toarray()

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [91]:
encode_oneHot.get_feature_names_out()

array(['Sex_female', 'Sex_male'], dtype=object)

In [101]:
feature_names = encode_oneHot.get_feature_names_out()
sex_encoded = pd.DataFrame(sex_encoded_sparseTrue.toarray(), columns=feature_names) # if sparse True must .toarray()
sex_encoded.head()

,Sex_female,Sex_male
0,0.0,1.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


Dapat dilihat kalau default dari  `OneHotEncoder` `sparsebool default=True`

Jadi hasil encoder menghasilkan banyak nilai 0  <br>
yang dimana  huge memory savings atau memory untuk komputasi menjadi besar

Apa itu [sparse matrix](https://scipy-lectures.org/advanced/scipy_sparse/introduction.html#why-sparse-matrices) 

Maka untuk tidak menghasilkan spaces matris
<br>Sparse di parameter dijadikan `sparse=False`





In [94]:
encode_oneHot=OneHotEncoder(sparse=False)
sex_encoded_sparseFalse = encode_oneHot.fit_transform(target_encode)
sex_encoded_sparseFalse

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [79]:
encode_oneHot.get_feature_names_out()

array(['Sex_female', 'Sex_male'], dtype=object)

In [80]:
encode_oneHot.categories_

[array(['female', 'male'], dtype=object)]

In [100]:
feature_names = encode_oneHot.get_feature_names_out()
sex_encoded = pd.DataFrame(sex_encoded_sparseFalse, columns=feature_names)
sex_encoded.head()

,Sex_female,Sex_male
0,0.0,1.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


## Evalute 

In [117]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

model_logisticReg = make_pipeline(
    OneHotEncoder(handle_unknown="ignore",sparse=False), LogisticRegression(max_iter=500)
)

In [124]:
from sklearn.model_selection import cross_validate
cv_results_logiscticReg = cross_validate(model_logisticReg, feature_nonNumeric, target)
cv_results_logiscticReg

{'fit_time': array([0.22292924, 0.22006249, 0.27299523, 0.22176933, 0.10118365]),
 'score_time': array([0.01649928, 0.01831627, 0.00900078, 0.00909209, 0.00932622]),
 'test_score': array([0.81564246, 0.80337079, 0.79775281, 0.78089888, 0.80898876])}

In [113]:
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

model_dec_tree = make_pipeline(
    OneHotEncoder(handle_unknown="ignore",sparse=False),DecisionTreeClassifier()
)

In [123]:
from sklearn.model_selection import cross_val_score
cv_results_model_decTree = cross_validate(model_dec_tree, feature_nonNumeric, target,cv=5)
cv_results_model_decTree

{'fit_time': array([0.08355331, 0.08411145, 0.1454494 , 0.15018845, 0.14962482]),
 'score_time': array([0.00659752, 0.00632334, 0.00993824, 0.01055312, 0.02289176]),
 'test_score': array([0.84916201, 0.81460674, 0.83146067, 0.80337079, 0.80898876])}

In [125]:
scores = cv_results_model_decTree["test_score"]
print(f"The accuracy is: {scores.mean():.3f} ± {scores.std():.3f}")

The accuracy is: 0.822 ± 0.017


In [126]:
scores = cv_results_logiscticReg["test_score"]
print(f"The accuracy is: {scores.mean():.3f} ± {scores.std():.3f}")

The accuracy is: 0.801 ± 0.012


Maka dapat dilihat bahwa accuracy dari model terbaik dengan dataset Titanic adalah menggunakan `DecisionTreeClassifier`